<div style="text-align: left; font-family:Arial; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;">PLANT LEAF DISEASE DETECTION PROBLEM USING XGBOOST</div>

<div style="text-align: left; font-family:Arial; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 25px;">SONG SONG HÓA MÔ HÌNH XGBOOST CHO BÀI TOÁN PHÁT HIỆN BỆNH LÁ CÂY</div>

#### Thành viên:
- Hồng Nhất Phương - 20120165
- Phạm Quốc Hưng - 19120522

**Kế hoạch và phân công công việc:** [Bảng phân công]()

#### 1. GIỚI THIỆU ĐỀ TÀI

Song song hóa mô hình học máy là một cách để tăng tốc quá trình huấn luyện, tiết kiệm thời gian và có thể triển khai mô hình trên các tập dữ liệu lớn hơn. Vì lý do đó, nhóm đã quyết định chọn đề tài song song hóa mô hình học máy, cụ thể là mô hình XGBoost được sử dụng cho bài toán phát hiện bệnh ở lá cây. 

Kế hoạch song song hóa của nhóm là viết lại thuật toán XGBoost mà không cần gọi thư viện của nó, sau đó song song hóa những đoạn code có thể song song và thực hiện tối ưu hóa để tăng hiệu suất. Để hoàn thành dự án này, nhóm đã sử dụng thư viện numpy, numba và môi trường google colab để chạy mô hình.

#### 2. CÀI ĐẶT

Bài toán là quá trình dự đoán bệnh từ hình ảnh lá cây bằng thuật toán XGBoost và tận dụng các thuật toán song song và kĩ thuật cải tiến để tăng tốc quá trình. Bài toán bao gồm các giai đoạn sau:
- Thu thập dữ liệu
- Trích xuất đặc trưng
- Cài đặt mô hình XGBoost tuần tự, huấn luyện mô hình và dự đoán kết quả
- Song song hóa mô hình XGBoost, thực hiện tối ưu hóa, huấn luyện mô hình và dự đoán kết quả
- Tổng kết và so sánh kết quả

##### 2.1 THU THẬP DỮ LIỆU

Bài toán sử dụng bộ dữ liệu **Tomato leaf leaf disease detection dataset** trên [Kaggle](https://www.kaggle.com/datasets/kaustubhb999/tomatoleaf?fbclid=IwAR3QVtWoGaimTXgPXEpXzz7jPiFVIkDnJ6XfHvld06-xw1DvJ8u-jGxky0A) chứa 10.000 mẫu huấn luyện và 1000 mẫu dự đoán. Bộ dữ liệu có 10 nhãn bao gồm 1 nhãn khỏe mạnh (healthy) và 9 nhãn bệnh: 
- Tomato_mosaic_virus
- Target_Spot
- Bacterial_spot
- Tomato_Yellow_Leaf_Curl_Virus
- Late_blight
- Leaf_Mold
- Early_blight
- Spider_mites Two-spotted_spider_mite
- Septoria_leaf_spot

Toàn bộ dữ liệu được lưu trong folder [dataset](https://github.com/NhatPhuong2106/Applied_Parallel_Programming/tree/main/dataset), thông tin chi tiết về tập dữ liệu huấn luyện được ghi trong file [**understand_data.ipynb**](https://github.com/NhatPhuong2106/Applied_Parallel_Programming/blob/main/data_preprocessing/understand_data.ipynb)


Hiển thị ngẫu nhiên một bức ảnh của từng class 

In [1]:
#Import thư viện
import matplotlib.pyplot as plt
import cv2
import os

#Đọc ảnh
image_path = "dataset/train"
labels = os.listdir(image_path)

#Hiển thị ảnh
plt.figure(figsize=(17, 7))
for i, disease in enumerate(labels):
    plt.subplot(2, 5, i + 1)
    list_image = os.listdir(image_path + '/' + disease)
    image = cv2.imread(os.path.join(os.path.join(image_path, disease), list_image[21]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(disease, fontsize = 10)
    plt.axis("off")

##### 2.2 RÚT TRÍCH ĐẶC TRƯNG

Rút trích đặc trưng tạo ra những đặc trưng tách lớp tốt cho việc phân lớp mẫu. Quá trình thực hiện rút trích đặc trưng sẽ trải qua những bước sau:

##### 2.2.1 Phân đoạn hình ảnh sử dụng Color Detection (Image segmentation using Color Detection)

Được sử dụng để tách hình ảnh chiếc lá khỏi nền. Hình ảnh chiếc lá được lấy ra khỏi nền bằng cách sử dụng ranh giới màu hsv, ranh giới màu sắc sẽ mở rộng thay vì giới hạn nghiêm ngặt để có thể quét được mọi trường hợp, cho ra hình ảnh chiếc lá được tách hoàn toàn khỏi nền.

Ảnh minh họa cho việc phân đoạn hình ảnh

<img src="picture/img_seg.jpg"  />

##### 2.2.2 Sử dụng các Global feature descriptors để tiến hành trích xuất đặc trưng

Global feature descriptors thường được sử dụng trong việc truy xuất hình ảnh, phát hiện và phân loại đối tượng. Các đặc trưng toàn cục (Global features) mô tả toàn bộ hình ảnh để khái quát hóa toàn bộ đối tượng, chúng bao gồm biểu diễn màu sắc, mô tả hình dạng và đặc trưng kết cấu,...

Trích xuất các đặc trưng trên 3 bộ mô tả tính năng toàn cục (global feature descriptors):
- Hình dạng: Hu Moments - được sử dụng để mô tả hình dạng của một đối tượng trong hình ảnh.
  
- Kết cấu: Haralick feature - được sử dụng để mô tả kết cấu của hình ảnh. Kết cấu đề cập đến hình thức, tính nhất quán hoặc “cảm giác” của bề mặt.
  
- Màu sắc: Color Histograms - đếm số lần cường độ điểm ảnh nhất định (hoặc phạm vi cường độ điểm ảnh) xuất hiện trong một hình ảnh. Bằng cách sử dụng biểu đồ màu, chúng ta có thể biểu thị sự phân bố thực tế hoặc “số lượng” của từng màu trong ảnh

Sau khi đã trích xuất, kết quả sẽ được nối lại để tạo thành một mảng duy nhất.

##### 2.2.3 Chuẩn hóa đặc trưng và lưu kết quả

Toàn bộ cách thực hiện chi tiết được ghi trong file [**feature_extract.ipynb**]() 